In [0]:
# import urllib
# access_key = "***"
# secret_key = "***"

# encoded_secret_key = urllib.parse.quote(secret_key, "")

# bucket_name = 'spa-bucket-group-231'
# mount_name = 's3_bucketdata_gr231'
# #dbutils.fs.mount("s3n://%s:%s@%s" % (access_key, encoded_secret_key, bucket_name), "/mnt/%s" % mount_name)



In [0]:
%sh

ls -lrt /dbfs/mnt/s3_bucketdata_gr231/location_data/

#ls -lrt /dbfs/mnt/s3_bucketdata_gr231/

total 0


In [0]:
import random
import math
import csv
import pandas as pd
import datetime
import time
 



In [0]:
# Constants for mall location (Bengaluru, India in this example)
MALL_LATITUDE = 12.978388
MALL_LONGITUDE = 77.594334
EARTH_RADIUS_KM = 6371.0  # Radius of the Earth in kilometers
RADIUS_KM = 5.0  # Desired radius in kilometers

 



In [0]:
# Read data from the CSV file generated in Ex:1
pizza_customer_cluster_df = pd.read_csv("/dbfs/mnt/s3_bucketdata_gr231/clustering_data/pizzacustomer_segmented.csv")

 



In [0]:
# customer ids and label store them in a list
#grouped = pizza_customer_cluster_df.groupby['CustomerID']
grouped = list(zip(pizza_customer_cluster_df['CustomerID'], pizza_customer_cluster_df['cluster3']))
#print(grouped)

 



In [0]:
# Function to calculate the Haversine distance between two points in kilometers
def calculate_haversine_distance(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

 

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = EARTH_RADIUS_KM * c

    distance = distance

 

    return distance

 



In [0]:
# Function to generate random coordinates within a specified radius
def generate_random_coordinates():
    # Generate a random angle (in radians) for the direction
    random_angle = random.uniform(0, 2 * math.pi)

 

    # Generate a random distance within the specified radius
    random_distance = random.uniform(0, RADIUS_KM / EARTH_RADIUS_KM)

 

    # Calculate new latitude and longitude
    new_latitude = MALL_LATITUDE + (random_distance * math.cos(random_angle))
    new_longitude = MALL_LONGITUDE + (random_distance * math.sin(random_angle))

 

    return round(new_latitude, 6), round(new_longitude, 6)

 



In [0]:
# Generate coordinates for customers
def generate_customer_location():
   # Generate coordinates for customers and store them in a list
    customer_data = []

    # Iterate through the grouped data and append to the customer_data list
    for item in grouped:
        customer_id, cluster3 = item
        customer_lat, customer_lon = generate_random_coordinates()
        distance_to_mall = calculate_haversine_distance(MALL_LATITUDE, MALL_LONGITUDE, customer_lat, customer_lon)
        customer_data.append([customer_id, cluster3,customer_lat,customer_lon,distance_to_mall])

 

    # Write customer data to a CSV file
    cts = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    cust_location_filename='/dbfs/mnt/s3_bucketdata_gr231/location_data/customer_location_data_'+cts+'.csv'
    with open(cust_location_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['customer_id', 'cluster3', 'latitude', 'longitude', 'distance_to_mall_km'])
        writer.writerows(customer_data)



In [0]:
def generate_realtime_data():
    while True:
        generate_customer_location()
       # Wait for a random interval of 10 mins to simulate near real-time data
        time.sleep(600)



In [0]:
generate_realtime_data()